In [340]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
import functools
from xgboost import XGBClassifier, XGBRegressor
import numpy as np

load_dotenv()

TBA_API_KEY = os.getenv("TBA_API_KEY")
TBA_HEADER = {"X-TBA-Auth-Key": TBA_API_KEY}

In [341]:

FEATURE_ORDER = [
    'week',
    'red3_epa', 'red2_epa', 'red1_epa',
    'blue3_epa', 'blue2_epa', 'blue1_epa',
    'red3_total_points', 'red2_total_points', 'red1_total_points',
    'blue3_total_points', 'blue2_total_points', 'blue1_total_points',
    'red3_auto_points', 'red2_auto_points', 'red1_auto_points',
    'blue3_auto_points', 'blue2_auto_points', 'blue1_auto_points',
    'red3_teleop_points', 'red2_teleop_points', 'red1_teleop_points',
    'blue3_teleop_points', 'blue2_teleop_points', 'blue1_teleop_points',
    'red3_endgame_points', 'red2_endgame_points', 'red1_endgame_points',
    'blue3_endgame_points', 'blue2_endgame_points', 'blue1_endgame_points',
    'red3_rank', 'red2_rank', 'red1_rank',
    'blue3_rank', 'blue2_rank', 'blue1_rank',
    'red3_winrate', 'red2_winrate', 'red1_winrate',
    'blue3_winrate', 'blue2_winrate', 'blue1_winrate',
    'red3_coral_count', 'red2_coral_count', 'red1_coral_count',
    'blue3_coral_count', 'blue2_coral_count', 'blue1_coral_count',
    'red3_l4_count', 'red2_l4_count', 'red1_l4_count',
    'blue3_l4_count', 'blue2_l4_count', 'blue1_l4_count',
    'red3_l3_count', 'red2_l3_count', 'red1_l3_count',
    'blue3_l3_count', 'blue2_l3_count', 'blue1_l3_count',
    'red3_algae_count', 'red2_algae_count', 'red1_algae_count',
    'blue3_algae_count', 'blue2_algae_count', 'blue1_algae_count',
    'red3_opr', 'red2_opr', 'red1_opr',
    'blue3_opr', 'blue2_opr', 'blue1_opr',
    'red3_ccwm', 'red2_ccwm', 'red1_ccwm',
    'blue3_ccwm', 'blue2_ccwm', 'blue1_ccwm'
]

In [342]:
def get_sb_team_stats_event(team, event_key):
    
    try:
        req = requests.get(f"https://api.statbotics.io/v3/team_year/{str(team)}/{event_key[:4]}")
        
        team_dict = req.json()
        
        team_info = {
            "event": event_key,
            "team": team_dict["team"],
            "epa": team_dict["epa"]["norm"],
            "total_points": team_dict["epa"]["breakdown"]["total_points"],
            "auto_points": team_dict["epa"]["breakdown"]["auto_points"],
            "teleop_points": team_dict["epa"]["breakdown"]["teleop_points"],
            "endgame_points": team_dict["epa"]["breakdown"]["endgame_points"],
            "winrate": team_dict["record"]["winrate"],
            "rank": team_dict["epa"]["ranks"]["total"]["rank"],
            
            }
        
        return team_info
    except requests.ConnectionError as e:
        print(e)
    except KeyError as e:
        raise KeyError(f"Key Error fetching statbotics stats\n{team}, {event_key}\n{e}")

In [343]:
def get_tba_oprs_event(event_key):
    try:
        req = requests.get(f"https://www.thebluealliance.com/api/v3/event/{event_key}/oprs", TBA_HEADER)
        
        oprs_res = req.json()
        
        req = requests.get(
            f"https://www.thebluealliance.com/api/v3/event/{event_key}/coprs",
            TBA_HEADER,
        )

        coprs_res = req.json()
        final_oprs = {}
        final_oprs = {
            "opr": oprs_res['oprs'], 
            "ccwm": oprs_res['ccwms'],
            "l3_count": coprs_res["L3 Coral Count"], 
            "l4_count": coprs_res["L4 Coral Count"],
            "coral_count": coprs_res["Total Coral Count"],
            "algae_count": coprs_res["Total Algae Count"]
            }
        
        return final_oprs
    except requests.ConnectionError as e:
        print(e)
    except KeyError as e:
        raise KeyError(f"Key Error fetching TBA stats {event_key}\n{e}")
    

In [ ]:
get_tba_oprs_event("2025ohsc")

In [345]:
@functools.lru_cache(maxsize=64)
def get_tba_oprs_team_event(team, event_key):
    team = str(team)
    oprs_info = get_tba_oprs_event(event_key)
    
    team_specific_stats = {}
    
    for opr in oprs_info.keys():
        team_specific_stats[opr] = oprs_info[opr][f"frc{str(team)}"]
        
    return team_specific_stats

In [ ]:
get_tba_oprs_team_event("4028", "2025ohsc")

In [347]:
def get_team_features(team, event_key):
    team = str(team)
    sb_stats = get_sb_team_stats_event(team, event_key)
    
    tba_stats = get_tba_oprs_team_event(team, event_key)
    
    return sb_stats | tba_stats

In [ ]:
get_team_features(4028, "2025ohsc")

In [349]:
def get_event_week(event_key):
    try:
        req = requests.get(f"https://www.thebluealliance.com/api/v3/event/{event_key}", TBA_HEADER)
        
        res = req.json()
        
        return res['week']
    except requests.ConnectionError as e:
        print(f"Error fetching {event_key} week:\n{e}")

In [350]:
get_event_week("2025mxle")

2

In [360]:
def get_match_features(match_key: str) -> dict:

    event_key = match_key.split("_")[0]

    try:
        req = requests.get(
            f"https://www.thebluealliance.com/api/v3/match/{match_key}/simple",
            TBA_HEADER,
        )
        req.raise_for_status()
        alliances = req.json()["alliances"]

        red_teams = [team[3:] for team in alliances["red"]["team_keys"]]
        blue_teams = [team[3:] for team in alliances["blue"]["team_keys"]]

        event_week = get_event_week(event_key)

        raw_features = {}
        raw_features['week'] = 8 if event_week is None else event_week

        for i in range(3):
            
            red_team_stats = get_team_features(red_teams[i], event_key)
            blue_team_stats = get_team_features(blue_teams[i], event_key)

            for stat_name, value in red_team_stats.items():

                if stat_name not in ["team", "event"]:
                    raw_features[f"red{i+1}_{stat_name}"] = value

            for stat_name, value in blue_team_stats.items():
                if stat_name not in ["team", "event"]:
                    raw_features[f"blue{i+1}_{stat_name}"] = value

        ordered_match_features = {
            feature: raw_features.get(feature, 0.0) for feature in FEATURE_ORDER
        }

        return ordered_match_features

    except requests.exceptions.RequestException as e:
        print(f"Error obteniendo datos para el partido {match_key}: {e}")
        return None
    except KeyError as e:
        print(
            f"Error procesando el partido {match_key}. ¿Falta un equipo o una clave?: {e}"
        )
        return None

In [353]:

classifier = XGBClassifier()
classifier.load_model('../models/classification/classification.json')

In [354]:
red_regressor = XGBRegressor()
red_regressor.load_model('../models/regression/red_model.json')

In [355]:
blue_regressor = XGBRegressor()
blue_regressor.load_model('../models/regression/blue_model.json')

In [361]:
def predict_match_outcome(match_key: str):
    features_dict = get_match_features(match_key)
    
    if not features_dict:
        print(f"Error: No se pudieron obtener las características para el partido {match_key}.")
        return None

    features_df = pd.DataFrame([features_dict])

    try:
        win_probabilities = classifier.predict_proba(features_df)[0]
        
        prob_red_win = win_probabilities[0]
        prob_blue_win = win_probabilities[1]
        
        predicted_winner = "blue" if prob_blue_win > prob_red_win else "red"

        predicted_red_score = red_regressor.predict(features_df)[0]
        predicted_blue_score = blue_regressor.predict(features_df)[0]

        
        prediction_result = {
            "match_key": match_key,
            "predicted_winner": predicted_winner,
            "win_probability": {
                "red": round(float(prob_red_win), 4),
                "blue": round(float(prob_blue_win), 4)
            },
            "predicted_scores": {
                "red": int(round(predicted_red_score)),
                "blue": int(round(predicted_blue_score))
            }
        }
        
        return prediction_result

    except Exception as e:
        print(f"Ocurrió un error durante la predicción para {match_key}: {e}")
        return None

In [372]:
match_to_predict = "2025cafb_qm6" 
full_prediction = predict_match_outcome(match_to_predict)

if full_prediction:
    print(f"--- Predicción para {full_prediction['match_key']} ---")
    print(f"Ganador Predicho: {full_prediction['predicted_winner'].upper()}")
    print(f"  Probabilidad de Victoria:")
    print(f"    - Azul: {full_prediction['win_probability']['blue']:.2%}")
    print(f"    - Rojo: {full_prediction['win_probability']['red']:.2%}")
    print(f"  Puntajes Estimados:")
    print(f"    - Azul: {full_prediction['predicted_scores']['blue']}")
    print(f"    - Rojo: {full_prediction['predicted_scores']['red']}")

--- Predicción para 2025cafb_qm6 ---
Ganador Predicho: RED
  Probabilidad de Victoria:
    - Azul: 23.59%
    - Rojo: 76.41%
  Puntajes Estimados:
    - Azul: 100
    - Rojo: 108
